In [ ]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent / "src"))

import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
commits = pd.read_csv('commits.csv', index_col=0, parse_dates=['date'])
df = commits.sort_values(by='date').reset_index().drop(columns=['index'])

In [ ]:
from private import email_dict
df = df.replace({'email': email_dict})

In [ ]:
df = df.loc[~df["path"].astype(str).str.endswith("package-lock.json")]
df = df.loc[~df["path"].astype(str).str.endswith("custom.css")]
df = df.loc[~df["path"].astype(str).str.endswith("custom.css.map")]
df = df.loc[~df["path"].astype(str).str.endswith("realm.json")]
df = df.loc[~df["path"].astype(str).str.endswith("gradlew")]
df = df.loc[~df["path"].astype(str).str.endswith("gradlew.bat")]
df = df.loc[~df["path"].astype(str).str.endswith(".png")]
df = df.loc[~df["path"].astype(str).str.endswith(".jar")]
df = df.loc[~df["path"].astype(str).str.startswith("build/")]

In [ ]:
df['yw'].min()

In [ ]:
df['weight'] = df['ins'] + df['del']
df['yw'] = df['date'].dt.isocalendar().year * 100 + df['date'].dt.isocalendar().week

df1 = df.drop(columns=['date', 'ins', 'del', 'dir', 'path']).groupby(['email', 'yw']).sum().reset_index()

In [ ]:
# Fill empty indexes
emails = df1['email'].unique()
weeks = [list(range(100*y+1, 100*y+53)) for y in range(2009, 2024)]
weeks = [item for sublist in weeks for item in sublist]
index_template = pd.MultiIndex.from_product([emails, weeks], names=["email", "yw"])
df_template = pd.DataFrame(index = index_template).reset_index()
df_template['zero'] = 0
df2 = df_template.set_index(keys=['email', 'yw']).join(df1.set_index(keys=['email', 'yw'])).fillna(0)
df2['weight'] = df2['weight'] + df2['zero']
df2 = df2.reset_index()

In [ ]:
import datetime
df2['year'] = df2['yw'] // 100
df2['week'] = df2['yw'] % 100
df2['date'] = df2.apply(lambda row: datetime.datetime.strptime("{}-{}-7".format(row['year'], row['week']), "%G-%V-%u"), axis=1)

In [ ]:
df2['weight'] = df2['weight'].clip(upper=2000)

In [ ]:
df2['moving'] = df2.groupby(['email']).rolling(2, 1)['weight'].mean().droplevel(level=[0])

In [ ]:
df2 = df2.loc[df2['yw'] > 202200]
df2 = df2.loc[df2['yw'] < 202310]

In [ ]:
fig = px.line(df2, x='date', y='moving', color='email', markers=True)
fig.show()

In [ ]:
fig = px.line(df2, x='date', y='moving', color='email', markers=True)
fig.show()